In [9]:
import io
from collections import namedtuple
from PIL import Image, ImageDraw, ImageFont
%matplotlib inline
import matplotlib.pyplot as plt
import json
import numpy as np
from google.protobuf.json_format import MessageToJson
from google.protobuf.json_format import Parse
from google.protobuf import json_format
import os
import sys
sys.path.append("..")
from models.bounding_box import FeatureType, Point, BoundingBox, DSU
from utils.cv_preprocess import draw_boxes, group_bounding_boxes, merge_box_groups

from utils.file_utils import prepare_image_local, prepare_image_web, load_counter, save_counter, save_json

from utils.nlp_preprocess import load_spacy_models, is_english, is_chinese, split_dish_info, is_word_relevant
from models.word_unit import WordUnit
from models.dish_segmenter import Dish
from IPython.display import display



In [10]:
def crop_image_to_segments(img_path, min_dimension=900):
    # Load the image
    with Image.open(img_path) as img:
        img_width, img_height = img.size

        # Determine the number of segments to crop based on the image dimensions
        # If width or height is less than 1000, we will not enter the loop and will save only the original image
        segments = []
        if img_width >= img_height:  # Landscape or square orientation
            for i in range(0, img_width, min_dimension):
                left = i
                upper = 0
                right = min(i + min_dimension, img_width)
                lower = img_height
                segments.append((left, upper, right, lower))
        else:  # Portrait orientation
            for i in range(0, img_height, min_dimension):
                left = 0
                upper = i
                right = img_width
                lower = min(i + min_dimension, img_height)
                segments.append((left, upper, right, lower))

        # Crop and save each segment
        for idx, (left, upper, right, lower) in enumerate(segments):
            cropped_img = img.crop((left, upper, right, lower))
            # Create a new filename for each segment
            filename, ext = os.path.splitext(img_path)
            new_filename = f"{filename}_segment_{idx}{ext}"
            cropped_img.save(new_filename)
            print(f"Saved cropped segment to {new_filename}")

In [11]:
dir_path = '../downloaded_menu/cleaned_img'
file_filter = '13._Phoenix_Palace_1_page_1.png'
all_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]

file_list = []
# Find the first unprocessed file
for file_name in all_files:
    if file_name.startswith(file_filter):
        file_list.append(os.path.join(dir_path, file_name))


In [12]:
for file_path in file_list:
    crop_image_to_segments(file_path)

Saved cropped segment to ../downloaded_menu/cleaned_img\13._Phoenix_Palace_1_page_1_segment_0.png
Saved cropped segment to ../downloaded_menu/cleaned_img\13._Phoenix_Palace_1_page_1_segment_1.png
Saved cropped segment to ../downloaded_menu/cleaned_img\13._Phoenix_Palace_1_page_1_segment_2.png
